In [1]:
config = {
    'lr': 0.001,
    'epochs': 10,
    "batch_accumulation": 1,
    'fish_types': ["Torsk", "Sei", "Hyse"],
    'gear_types': ["Trål"],
    'mode': "regression",
    "sequence_len": 50,
    'log_target': False,
    "thresholds": [1000, 1000, 1000],
    "criterion": "mse",
    "optimizer": "adam",
    'model_params': {
        "input_dim": 10,
        "hidden_dim": [64, 3],
        "kernel_size": (3, 3),
        "num_layers": 2,
        "batch_first": False,
        "return_all_layers": False,
    },
    'ocean_data_path': '../data/copernicus/datasets/norway.nc',
    'route_data_path': '../data/VMS_DCA_joined/catch_routes.parquet'
}

In [2]:
from train_LSTM import train

model = train(config, run_name='Gaussian_filter_LSTM')

[  1,   200] | loss: 1884524010.587074
[  1,   400] | loss: 274795.002388
[  1,   600] | loss: 280841.029160
[  1,   800] | loss: 282447.831588
[  1,  1000] | loss: 264344.212363
[  1,  1200] | loss: 278756.259708
[  1,  1400] | loss: 263024.546207
[  1,  1600] | loss: 268275.235081
[  1,  1800] | loss: 281745.280012
[  1,  2000] | loss: 274857.361371
[  1,  2200] | loss: 259561.272856
[  1,  2400] | loss: 265245.112563
[  1,  2600] | loss: 268102.557866
[  1,  2800] | loss: 284533.971747
[  1,  3000] | loss: 276943.820339
[  1,  3200] | loss: 258219.020650
[  1,  3400] | loss: 275703.411009
[  1,  3561] | train loss: 262611.997799 | val loss: 264781.229374
[  2,   200] | loss: 283510.025430
[  2,   400] | loss: 251454.098085
[  2,   600] | loss: 256262.381702
[  2,   800] | loss: 248952.321082
[  2,  1000] | loss: 283504.078170
[  2,  1200] | loss: 266259.054344
[  2,  1400] | loss: 265923.233570
[  2,  1600] | loss: 268179.865510
[  2,  1800] | loss: 260104.171874
[  2,  2000] | loss

KeyboardInterrupt: 

In [2]:
from utils.dataloader import ForSeaDataset

dataset = ForSeaDataset(
    config["ocean_data_path"],
    config["route_data_path"],
    fish_types=config["fish_types"],
    gear_types=config["gear_types"],
    mode=config["mode"],
    sequence_len=config["sequence_len"],
    log_target=config["log_target"],
    batch_size=1,
    shuffle=True,
)

In [3]:
for a,b,c in dataset.train_data():
    print(a.shape, b.shape, c.shape)

ValueError: could not broadcast input array from shape (30,140,100) into shape (0,140,100)

In [4]:
dataset.ocean_data_shape[2:]

(10, 140, 100)

In [5]:
from models.GaussianFilterLSTM import ConvLSTM, LSTMAutoEncoder

out_dim = len(config["fish_types"])
# model = ConvLSTM(10, [64, 64, out_dim], (3,3), num_layers=3, batch_first=False, return_all_layers=False).cuda()
model = LSTMAutoEncoder((out_dim, *dataset.ocean_data_shape[3:]), dataset.land_mask, 10, [64, 64, out_dim], (3,3), num_layers=3, batch_first=False, return_all_layers=False).cuda()

In [6]:
model.eval()

pred = model(ocean, route)

torch.Size([3, 140, 100]) torch.Size([86, 2])


In [8]:
pred.shape

torch.Size([86, 3])

In [35]:
out[0].shape

torch.Size([1, 20, 3, 140, 100])